# Evaluate Hand-written Text Recognition

In [ ]:
from Levenshtein import distance
import os
import re
import sys
sys.path.append(os.getcwd() + '/..')
from scripts import read_transkribus_files

In [ ]:
def print_with_color(string, color_code=1):
    print(f"\x1b[3{color_code}m{string}\x1b[m", end="")

## 1. Estimate HTR accuracy by exmanining printed text

In [ ]:
PRINTED_TEXT = {}

PRINTED_TEXT[1831] = [
    "No",
    "Verbeteringen",
    "Heden den",
    "Compareerden voor mij",
    "van den Burgerlijken stand op dit eiland.",
    "De personen van",
    "van competenten ouderdom en alhier woonachtig.",
    "Dewelke aangifte hebben gedaan dat",
    "op den",
    "te",
    "ure des",
    "overleden is:",
    "Oud",
    "Geboren",
    "op den"
    "zijnde"
    "laatstelijk gewoond",
    "En is deze behoorlijk geteekend",
    "op Curacao dato utsupra.",
    "F",
    "Aanteekeningen"
]

PRINTED_TEXT[1842] = [
    "No",
    "Verbeteringen",
    "Heden den",
    "Compareerden voor mij",
    "van den Burgerlijken stand op dit eiland.",
    "De personen van",
    "van competenten ouderdom en alhier woonachtig.",
    "Dewelke aangifte hebben gedaan dat",
    "op den",
    "te",
    "ure des",
    "overleden is:",
    "Oud",
    "Geboren",
    "op den",
    "laatstelijk gewoond",
    "En is deze behoorlijk geteekend",
    "op Curacao dato utsupra.",
    "F",
    "Aanteekeningen"
]

PRINTED_TEXT[1842] = [
    "No",
    "Verbeteringen",
    "Heden den",
    "des Jaars",
    "Een duizend acht honderd en",
    "Compareerden voor mij",
    "van den Burgerlijken stand op dit eiland.",
    "De personen van",
    "van competenten ouderdom en alhier woonachtig, dewelke aangifte hebben gedaan dat",
    "op den",
    "des Jaars Een duizend acht honderd en",
    "te",
    "ure des",
    "overleden is:",
    "Oud",
    "Geboren",
    "op den",
    "laatstelijk gewoond",
    "En is deze behoorlijk geteekend",
    "op Curacao dato utsupra.",
    "F",
    "Aanteekeningen"
]

PRINTED_TEXT[1869] = [
    "Folio",
    "No",
    "Heden den",
    "des Jaars",
    "Een duizend acht honderd en",
    "Compareerden ten burele voor mij",
    "ambtenaar van den Burgerlijken stand in het",
    "district dezes eilands.",
    "De personen van",
    "van beroep",
    "oud",
    "wonende",
    "en",
    "van beroep",
    "oud",
    "wonende"
    "dewelke aangifte hebben gedaan",
    "op den",
    "des Jaars Een duizend acht honderd en",
    "te",
    "ure des",
    "overleden is:",
    "in den ouderdom van",
    "jaren",
    "van",
    "zijnde geboren",
    "En is hiervan akte opgemaakt, welke na voorlezing door de",
    "en mij",
    "ambtenaar",
    "voormeld is ondertekendteekend",
    "op Curacao dato utsupra.",
]

PRINTED_TEXT[1873] = [
    "Folio",
    "No",
    "Heden den",
    "des Jaars",
    "Een duizend acht honderd en",
    "Compareerden ten burele voor mij",
    "ambtenaar van den Burgerlijken stand in het",
    "district dezes eilands.",
    "De personen van",
    "van beroep",
    "oud",
    "wonende",
    "en",
    "van beroep",
    "oud",
    "wonende"
    "dewelke aangifte hebben gedaan",
    "op den",
    "des Jaars Een duizend acht honderd en",
    "te",
    "ure des",
    "overleden is:",
    "in den ouderdom van",
    "jaren",
    "van",
    "zijnde geboren",
    "En is hiervan akte opgemaakt, welke na voorlezing door de",
    "en mij ambtenaar voormeld",
    "is onderteekend",
    "op Curacao dato utsupra.",
]

PRINTED_TEXT[1885] = [
    "Folio",
    "No",
    "Heden den",
    "des Jaars",
    "Een duizend acht honderd en",
    "Compareerden ten burele voor mij",
    "ambtenaar van den Burgerlijken stand in het",
    "district dezes eilands.",
    "De personen van",
    "van beroep",
    "oud",
    "wonende",
    "en",
    "van beroep",
    "oud",
    "wonende"
    "dewelke aangifte hebben gedaan dat in het",
    "district alhier op den",
    "des Jaars Een duizend acht honderd en",
    "te",
    "ure des",
    "overleden is:",
    "in den ouderdom van",
    "jaren,",
    "van",
    "zijnde geboren",
    "En is hiervan akte opgemaakt, welke na voorlezing door de",
    "en mij ambtenaar voormeld",
    "is ondertekeend",
    "op Curacao dato utsupra.",
]


PRINTED_TEXT[1892] = [
    "Folio",
    "No",
    "Heden den",
    "des Jaars",
    "Een duizend acht honderd en",
    "en negentig",
    "Compareerden ten burele voor mij",
    "ambtenaar van den Burgerlijken stand in het",
    "district dezes eilands.",
    "De personen van",
    "van beroep",
    "oud",
    "jaren,",
    "wonende alhier",
    "en",
    "van beroep",
    "oud"
    "jaren,",
    "wonende alhier",
    "dewelke aangifte hebben gedaan dat in het",
    "district alhier op den",
    "des Jaars Een duizend acht honderd en",
    "en negentig",
    "te",
    "ure des",
    "overleden is:",
    "in den ouderdom van",
    "jaren,",
    "van",
    "zijnde geboren",
    "En is hiervan akte opgemaakt, welke na voorlezing door de",
    "en mij ambtenaar voor-",
    "meld is onderteekend",
    "op Curacao dato utsupra.",
]

PRINTED_TEXT[1894] = [
    "Folio",
    "No",
    "Heden den",
    "des Jaars",
    "Een duizend acht honderd en",
    "en negentig",
    "compareerden ten burele voor mij",
    "ambtenaar van den Burgerlijken stand in het",
    "district dezes eilands.",
    "De personen van",
    "van beroep",
    "oud",
    "jaren,",
    "wonende alhier",
    "en",
    "van beroep",
    "oud"
    "jaren,",
    "wonende alhier",
    "dewelke aangifte hebben gedaan dat in het",
    "district alhier op den",
    "des Jaars Een duizend acht honderd en",
    "en negentig",
    "te",
    "ure des",
    "overleden is:",
    "in den ouderdom van",
    "jaren,",
    "van",
    "zijnde geboren",
    "En is hiervan akte opgemaakt, welke na voorlezing door de",
    "en mij ambtenaar voor-",
    "meld is onderteekend",
    "op Curacao dato utsupra.",
]

PRINTED_TEXT[1895] = [
    "Folio",
    "No",
    "Heden den",
    "des Jaars",
    "Een duizend acht honderd en",
    "en negentig",
    "compareerden ten burele voor mij",
    "ambtenaar van den Burgerlijken stand in het",
    "district dezes eilands.",
    "De personen van",
    "van beroep",
    "oud",
    "jaren,",
    "wonende alhier",
    "en",
    "van beroep",
    "oud"
    "jaren,",
    "wonende alhier",
    "dewelke aangifte hebben gedaan dat in het",
    "district alhier op den",
    "des Jaars Een duizend acht honderd en",
    "en negentig",
    "te",
    "ure des",
    "overleden is:",
    "in den ouderdom van",
    "jaren,",
    "van",
    "zijnde geboren",
    "En is hiervan akte opgemaakt, welke na voorlezing door de",
    "en mij ambtenaar voor-",
    "meld is onderteekend",
    "op Curacao dato utsupra.",
]

PRINTED_TEXT[1900] = [
    "Folio",
    "No",
    "Heden den",
    "des Jaars",
    "Een duizend negen honderd",
    "compareerde voor mij",
    "ambtenaar van den Burgerlijken stand in het",
    "district dezes eilands ten mijnen bureele:",
    "De personen van",
    "van beroep",
    "oud",
    "jaren,",
    "wonende alhier",
    "en",
    "van beroep",
    "oud"
    "jaren,",
    "wonende alhier",
    "dewelke aangifte hebben gedaan dat in het",
    "district alhier op den",
    "des Jaars Een duizend negen honderd",
    "te",
    "ure des",
    "overleden is:",
    "in den ouderdom van",
    "jaren,",
    "van",
    "zijnde geboren",
    "En is hiervan acte opgemaakt, welke na voorlezing door de",
    "en mij ambtenaar voor-",
    "meld is onderteekend op Curacao dato utsupra.",
]

PRINTED_TEXT[1907] = [
    "Folio",
    "No",
    "Heden den",
    "des Jaars",
    "Een duizend negen honderd",
    "compareerden",
    "voor mij ambtenaar van den Burgerlijken stand in het",
    "district dezes eilands ten mijnen bureele:",
    "De personen van",
    "van beroep",
    "oud",
    "jaren,",
    "wonende alhier",
    "en",
    "van beroep",
    "oud"
    "jaren,",
    "wonende alhier",
    "dewelke aangifte hebben gedaan dat in het",
    "district alhier op den",
    "des Jaars Een duizend negen honderd",
    "te",
    "ure des",
    "overleden is:",
    "in den ouderdom van",
    "jaren,",
    "van",
    "zijnde geboren",
    "En is hiervan acte opgemaakt, welke na voorlezing door de",
    "en mij ambtenaar voormeld",
    "is onderteekend op Curacao dato utsupra.",
]

PRINTED_TEXT[1909] = [
    "Folio",
    "No",
    "Heden den",
    "des Jaars",
    "Een duizend negen honderd",
    "compareerden",
    "voor mij",
    "ambtenaar van den Burgerlijken stand in het",
    "district dezes eilands ten mijnen bureele:",
    "De personen van",
    "van beroep",
    "oud",
    "jaren,",
    "wonende alhier",
    "en",
    "van beroep",
    "oud"
    "jaren,",
    "wonende alhier",
    "dewelke aangifte hebben gedaan dat in het",
    "district alhier op den",
    "des Jaars Een duizend negen honderd",
    "te",
    "ure des",
    "overleden is:",
    "in den ouderdom van",
    "jaren,",
    "van",
    "zijnde geboren",
    "En is hiervan acte opgemaakt, welke na voorlezing door de",
    "en mij ambtenaar",
    "voormeld is onderteekend op Curacao dato utsupra.",
]

PRINTED_TEXT[1925] = [
    "Folio",
    "No",
    "Heden den",
    "des Jaars",
    "Een duizend negen honderd",
    "compareerden",
    "voor mij hulpambtenaar van den Burgerlijken stand in het",
    "district dezes eilands ten mijnen bureele:",
    "De personen van",
    "van beroep",
    "oud",
    "jaren,",
    "wonende alhier in het ",
    "district",
    "en",
    "van beroep",
    "oud"
    "jaren,",
    "wonende alhier in het",
    "district",
    "dewelke aangifte hebben gedaan dat in het",
    "district alhier op den",
    "des Jaars Een duizend negen honderd",
    "te",
    "ure des",
    "overleden is:",
    "in den ouderdom van",
    "jaren,",
    "van",
    "zijnde geboren",
    "En is hiervan acte opgemaakt, welke, na voorlezing door de",
    "en mij hulpambtenaar",
    "voormeld is onderteekend op Curacao dato utsupra.",
]

PRINTED_TEXT[1935] = [
    "Folio",
    "No",
    "Heden den",
    "negentienhonderd",
    "verscheen voor mij, hulpambtenaar van den Burgerlijken Stand",
    "in het",
    "district op Curaçao:",
    "oud",
    "jaren, van beroep",
    "wonende in het",
    "district alhier die op grond van eigen wetenschap verklaarde dat in het",
    "dis-"
    "trict alhier op den",
    "negentienhonderd",
    "ten",
    "ure des",
    "middags",
    "overleden is:",
    "in den ouderdom van",
    "Waarvan akte, welke is voorgelezen.",
]

PRINTED_TEXT[1937] = [
    "Folio",
    "No",
    "Heden den",
    "negentienhonderd",
    "verscheen voor mij, hulpambtenaar van den Burgerlijken Stand",
    "in het",
    "district op Curaçao:",
    "oud",
    "jaren, van beroep",
    "wonende in het",
    "district alhier die op grond van eigen wetenschap verklaarde dat in het",
    "district alhier op den",
    "negentienhonderd",
    "ten",
    "ure",
    "des",
    "middags overleden is:",
    "in den ouderdom van",
    "Waarvan akte, welke is voorgelezen.",
]

PRINTED_TEXT[1938] = [
    "No",
    "Folio",
    "Heden den",
    "negentienhonderd",
    "verscheen voor mij", 
    "hulpambtenaar van den Burgerlijken Stand",
    "oud",
    "jaren,",
    "beroep,",
    "wonende in het",
    "district op",
    "die op grond van eigen wetenschap verklaarde",
    "dat in het",
    "district op",
    "op den",
    "negentienhonderd",
    "ten",
    "ure des",
    "middags overleden is:",
    "in den ouderdom van",
    "Waarvan akte, welke is voorgelezen.",
]

PRINTED_TEXT[1939] = [
    "Nr.",
    "1939",
    "Blad nummer",
    "Heden",
    "negentienhonderd negen en dertig verscheen voor mij, hulpambtenaar van",
    "den Burgerlijken Stand op Curaçao",
    "oud",
    "jaren,",
    "wonende alhier, die verklaarde, daarvan uit eigen wetenschap kennis",
    "dragende, dat op",
    "dezes jaars",
    "middags te",
    "uur, alhier is overleden",
    "Waarvan akte, welke is voorgelezen.",
]

#### 1940 and late: to be checked

In [ ]:
data_dir = "../../data/Overlijden/x-samples/three-columns-100/page"

texts, metadata, textregions = read_transkribus_files.read_files(data_dir)

In [ ]:
def cleanup_text(text):
    return " ".join(text.strip().split())

In [ ]:
BATCHSIZE = 1
PROMPT = """Here is a Dutch death certificate. 
Can you extract the death date from this document? 
Please present the result in numeric form, with a zero prepending day and month numbers smaller than 10, so for example like: 01-01-1900.
Only give the death date as result. 
Other information from the document is not necessary.
""" 
counter = 0
for key in sorted(texts.keys()):
    year, district, folio_nbr = key.split("-")
    print(PROMPT + re.sub("\n", " ", texts[key]))
    counter += 1
    if counter >= BATCHSIZE:
        counter = 0
        print("\n")

In [ ]:
def compare_strings(string1, text):
    clean_string = re.sub("[^a-z0-9]", "", re.sub("ç", "c", string.lower()))
    clean_text = re.sub("[^a-z0-9]", "", re.sub("ç", "c", text.lower()))
    return clean_string in clean_text

In [ ]:
def get_template_id(PRINTED_TEXT, year):
    template_id = 0
    for key in PRINTED_TEXT.keys():
        if key > template_id and key <= int(year):
            template_id = key
    return template_id

In [ ]:
scores = {}
for key in sorted(texts.keys()):
    year, district, folio_nbr = key.split("-")
    template_id = get_template_id(PRINTED_TEXT, year)
    correct_text = ""
    missed_text = ""
    for string in PRINTED_TEXT[template_id]:
        if compare_strings(string, texts[key]):
            correct_text += string
        else:
            missed_text += string
    scores[key] = len(correct_text)/len(correct_text + missed_text)
[(key[1], key[0]) for key in sorted(scores.items() ,key=lambda item: item[1])][-5:]

## 2. Compare names with gold data

In [ ]:
import pandas as pd

In [ ]:
def fix_year(text_id, date_in):
    print(text_id, date_in)
    target_year = int(text_id[:4])
    guessed_year = int(date_in[-4:])
    if guessed_year != target_year and guessed_year != target_year-1:
        print_with_color(f"changing year {guessed_year} to {target_year}!\n")
        date_out = date_in[0:-4] + str(target_year)
    else:
        date_out = date_in
    return date_out

In [ ]:
GOLD_DATA = "../../data/Overlijden/x-samples/three-columns-100.csv"

In [ ]:
gold_data = pd.read_csv(GOLD_DATA)

In [ ]:
GOLD_DATA = "../../data/Overlijden/x-samples/three-columns-100.csv"
gold_data = pd.read_csv(GOLD_DATA)
names = {}
for key in gold_data.index:
    names_key = read_transkribus_files.make_file_id(gold_data["scans"][key])
    if isinstance(gold_data["first_names"][key], str):
        if isinstance(gold_data["last_name"][key], str):
            name = str(gold_data["first_names"][key]) + " " + str(gold_data["last_name"][key])
        else:
            name = str(gold_data["first_names"][key])
    elif isinstance(gold_data["last_name"][key], str):
        name = str(gold_data["last_name"][key])
    if names_key in names:
        names[names_key].append(name)
    else:
        names[names_key] = [name]

In [ ]:
chatgpt_deceased_names = list(pd.read_csv("deceased_names_gpt35.csv", header=None)[0])

In [ ]:
counter = 0
names_correct = 0
name_distances = {}
for key in sorted(texts.keys()):
    name_correct = False
    levenshtein_minimum = 999999
    for name in names[key]:
        if not name_correct:
            name_correct = (chatgpt_deceased_names[counter].lower() == name.lower())
        levenshtein_distance = distance(chatgpt_deceased_names[counter].lower(), name.lower())
        if levenshtein_distance < levenshtein_minimum:
            levenshtein_minimum  = levenshtein_distance
    counter += 1
    if name_correct:
        names_correct += 1
    if levenshtein_minimum not in name_distances:
        name_distances[levenshtein_minimum] = 1
    else:
        name_distances[levenshtein_minimum] += 1
names_correct, name_distances

In [ ]:
date_of_death_gold = {}
for key in gold_data.index:
    date_of_death_gold_key = read_transkribus_files.make_file_id(gold_data["scans"][key])
    if isinstance(gold_data["date_of_death"][key], str):
        date_of_death_gold_value = gold_data["date_of_death"][key]
        if date_of_death_gold_key in date_of_death_gold:
            date_of_death_gold[date_of_death_gold_key].append(date_of_death_gold_value)
        else:
            date_of_death_gold[date_of_death_gold_key] = [date_of_death_gold_value]

In [ ]:
counter = 0
correct_death_date_found_count = 0
for text_id in sorted(texts.keys()):
    try:
        chatgpt_death_dates[counter] = fix_year(text_id, chatgpt_death_dates[counter])
        date_correct = chatgpt_death_dates[counter] == date_of_death_gold[text_id][0]
        print(counter, chatgpt_death_dates[counter], date_of_death_gold[text_id][0], date_correct)
        if date_correct:
            correct_death_date_found_count += 1 
    except:
        print_with_color(f"error for text id {text_id}!\n")
    counter += 1
    if counter >= len(chatgpt_death_dates):
        break
correct_death_date_found_count

to do: check for missing gold data